In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

d:\Agentic_AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
FILE_PATH = r"D:\Agentic_AI\2-Langchain\2.2-DataTransformation\speech.txt"

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(FILE_PATH)
docs = loader.load()

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

docs = splitter.split_documents(docs)

In [9]:
final_docs = []

for doc in docs:
    final_docs.append(doc.page_content)

final_docs

['The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.',
 'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…',
 'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a peo

In [10]:
embedded_query = embedding.embed_documents(final_docs)

In [12]:
embedded_query[0]

[-0.0012092484394088387,
 0.09181910753250122,
 0.002881293185055256,
 0.0061694723553955555,
 -0.02555806376039982,
 0.010189753957092762,
 -0.008642702363431454,
 -0.04298453778028488,
 0.05521494895219803,
 0.023454317823052406,
 0.06972502917051315,
 -0.00683547742664814,
 -0.030210956931114197,
 -0.04264739155769348,
 0.007255852222442627,
 -0.09552040696144104,
 0.043725088238716125,
 0.014230464585125446,
 0.029193131253123283,
 0.026581600308418274,
 -0.05027371272444725,
 0.007587359752506018,
 -0.023739660158753395,
 -0.03282768279314041,
 -0.02693922072649002,
 0.005367743782699108,
 -0.005853366572409868,
 -0.006293206010013819,
 0.03960249945521355,
 -0.003065861761569977,
 -0.0175885371863842,
 -0.017877819016575813,
 -0.008162781596183777,
 -0.07314983755350113,
 2.250705847473e-06,
 -0.0006386682507582009,
 0.029857931658625603,
 -0.004462303593754768,
 0.004164202604442835,
 -0.04127618670463562,
 0.010725096799433231,
 0.008400856517255306,
 -0.011143218725919724,
 -0

In [14]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [16]:
index = faiss.IndexFlatL2(768)

In [23]:
vector_store = FAISS(
    embedding_function=embedding,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [20]:
from langchain.schema import Document

# Example: convert strings to Documents
final_docs = [Document(page_content=doc) for doc in final_docs]

In [22]:
final_docs

[Document(metadata={}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={}, page_content='It will be all the easier for us to conduct ourselves as be

In [24]:
vector_store.add_documents(final_docs)

['ce290303-839f-42d5-9a2e-01682465baa8',
 'ff8a9c44-7768-4848-ac69-e3250f26e568',
 'b7456acd-eb3b-4122-8332-932085e3b899',
 'f2ac0a4e-0171-4da7-be47-4053fd1423b0',
 '176d00e0-4246-4e74-8ae5-513c34bd972b',
 '9619e6aa-2aef-40e3-9b54-9fa731e52277',
 '3c0b6f8c-1b66-4b27-8740-9addf2c44288',
 'ad7b0d76-e233-4722-a654-09ede30e466d',
 'f909923d-2fed-4712-ace0-f2f582c63252',
 '5da16b11-9c12-4eda-a65b-ddf1821fffe0']

In [25]:
results = vector_store.similarity_search("What was Wilson's view on the German people?", k=3)
results

[Document(id='b7456acd-eb3b-4122-8332-932085e3b899', metadata={}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early'),
 Document(id='176d00e0-4246-4e74-8ae5-513c34bd972b', metadata={}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German bi